In [2]:
# Assuming TensorFlow is already installed
# If not, 
pip install tensorflow

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import random

SyntaxError: invalid syntax (2258713680.py, line 3)

In [ ]:
# Corpus: Seed text for training
corpus = [
    "The wind was still. Nothing moved. Yet, everything had changed.",
    "Stillness was not silence. In the quiet, thoughts screamed louder.",
    "They met at the edge of knowing, unsure whether to return or leap.",
    "Leaps are lies we tell ourselves — until we land.",
    "Truth is a prism: beautiful, breakable, and bent by perspective.",
    "Sometimes, forgetting is a kind of freedom.",
    "Pain is memory’s way of proving you were alive.",
    "They stitched the world back together with whispers and wishes.",
    "Dreams refused to be caged in logic.",
    "Every echo was a version of herself, trying to return."
]

In [ ]:
# Tokenize
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

In [ ]:
# Create input sequences
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(2, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [ ]:
# Pad sequences
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Split into predictors and label
X, y = input_sequences[:,:-1], input_sequences[:,-1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [ ]:
# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words, 64, input_length=max_sequence_len - 1),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(total_words, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
# Train model (quick run; increase epochs for better results)
model.fit(X, y, epochs=100, verbose=1)

In [ ]:
# Generate text
def generate_text(seed_text, next_words=30, diversity=1.0):
    result = []
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predictions = model.predict(token_list, verbose=0)[0]
        
        # Add diversity
        predictions = np.asarray(predictions).astype('float64')
        predictions = np.log(predictions + 1e-8) / diversity
        exp_preds = np.exp(predictions)
        predictions = exp_preds / np.sum(exp_preds)
        predicted = np.random.choice(range(len(predictions)), p=predictions)

        output_word = ''
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += ' ' + output_word
        result.append(output_word)
    return seed_text

In [ ]:
# Example use
seed = "The sky cracked"
print("NeuroLit Output:\n")
print(generate_text(seed))